In [4]:
import mlflow
import numpy as np
from rfq_data_loader import load_data
from preprocesser import preprocesser
from client import client_avg, client_detail, client_weighted_hist_pref
from unique_ISIN import unique_ISIN
from sim_matrix import (
    sim_calculator,
    find_top_n_similar_bonds,
    is_newbond,
    isin_to_features_func,
    client_apetite,
    client_apetite_dict,
    bond_to_n_client,
)
from new_bond import new_bond_to_df, new_bond_to_vec

In [3]:
def global_run(file_path="RFQ_Data_Challenge_HEC.csv"):
    # Extract the raw data
    df = load_data(file_path)
    # Make a series of operations to the data to have a cleaned dataframe to work with
    df = preprocesser(df)

    # Get a dataframe with unique ISIN codes, with their returns and risks
    df_unique = unique_ISIN(df=df)

    # Create similarity matrix and preprocessors for future use
    similarity_matrix, scaler, one_hot_encoder, features = sim_calculator(
        df_unique=df_unique
    )
    # Create a dict to map ISIN codes to its features
    isin_to_features_dict = isin_to_features_func(
        df_unique=df_unique, features=features
    )

    # Create for each client the average bonds (specifying if we take only the 100 more recent transactions or all the transactions)
    client_apetite_dict_hist = client_apetite_dict(
        recent100=False, df=df, isin_to_features=isin_to_features_dict
    )
    client_apetite_dict_recent = client_apetite_dict(
        recent100=True, df=df, isin_to_features=isin_to_features_dict
    )

    return (
        isin_to_features_dict,
        client_apetite_dict_hist,
        client_apetite_dict_recent,
        features,
        df_unique,
    )

In [4]:
rfq_path = "../../RFQ_Data_Challenge_HEC.csv"
(
    isin_to_features_dict,
    client_apetite_dict_hist,
    client_apetite_dict_recent,
    features,
    df_unique,
) = global_run(file_path=rfq_path)

/Users/francoismoreau/Documents/Polytechnique/4A/Natixis/natixis_challenge/src/model_construction/rfq_data_loader.py:4: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/Users/francoismoreau/Documents/Polytechnique/4A/Natixis/natixis_challenge/src/model_construction/preprocesser.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["B_Price"] = pd.to_numeric(


In [5]:
# if there is an existing bond, run this cell
isin_code = "USY7S272AH57"
new_bond_features = isin_to_features_dict[isin_code].reshape(1, -1)

In [7]:
client_hist_new_bond = bond_to_n_client(
    bond_vect=new_bond_features, client_apetite_dict=client_apetite_dict_hist, n=10
)
client_recent_new_bond = bond_to_n_client(
    bond_vect=new_bond_features, client_apetite_dict=client_apetite_dict_recent, n=10
)

In [8]:
bonds_new_bond = find_top_n_similar_bonds(
    new_bond_features=new_bond_features,
    existing_features=features,
    df_unique=df_unique,
    n=10,
)

In [9]:
client_hist_new_bond

["Banque Populaire De L'Ouest",
 'Cegi Ett',
 'Ax',
 'Bred',
 'Bnp Paribas',
 'Sg',
 'Axa France Iard',
 'Ce Loire Centre',
 'Oso',
 'Banque Cantonale Du Jura']

In [10]:
client_recent_new_bond

['Coface',
 'Cegc',
 'Qbe',
 'Socamet',
 'Societe Generale Artenay',
 'Bnp Reunion',
 'License De Transport',
 'Maitre Bondroit',
 'Hsbc',
 'Ce Loire Centre']

In [11]:
bonds_new_bond

,ISIN,Coupon,BloomIndustrySubGroup,Classification,Country,Ccy,Rating_SP,Maturity,dl_maturity_years,Type,Rating_SP_Ordinal
634678,USY7S272AH57,5.875,Steel-Producers,Basic Materials,KOREA,USD,A-,2033-01-17,9.01,Fixed,15
82240,USY7S272AG74,5.750,Steel-Producers,Basic Materials,KOREA,USD,A-,2028-01-17,4.00,Fixed,15
93684,USY7S272AF91,5.625,Steel-Producers,Basic Materials,KOREA,USD,A-,2026-01-17,2.00,Fixed,15
20999,USY7S272AD44,4.375,Steel-Producers,Basic Materials,KOREA,USD,A-,2025-08-04,1.55,Fixed,15
188733,USY8085FBL32,6.500,Electronic Compo-Semicon,Technology,KOREA,USD,BBB-,2033-01-17,9.01,Fixed,12
119810,XS2703610050,6.875,Finance-Invest Bnkr/Brkr,Financials,KOREA,USD,BBB,2026-11-06,2.81,Fixed,13
418736,USY8085FBK58,6.375,Electronic Compo-Semicon,Technology,KOREA,USD,BBB-,2028-01-17,4.00,Fixed,12
423625,US500630EB00,5.625,Special Purpose Banks,Government,KOREA,USD,AA,2033-10-23,9.77,Fixed,19
404642,US45604HAN70,5.375,Special Purpose Banks,Financials,KOREA,USD,AA-,2028-10-04,4.72,Fixed,18
97653,US302154DW60,5.125,Export/Import Bank,Government,KOREA,USD,AA,2033-01-11,8.99,Fixed,19
